In [2]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModel

import torch
from torch.nn import functional as F

In [4]:
MODEL_NAME = 'deepset/sentence_bert'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at deepset/sentence_bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
data = pd.read_csv('drive/MyDrive/Biocreative/Biocreative/litcovid_dataset.csv')
data = data.drop(['pmid', 'journal', 'title', 'keywords', 'pub_type', 'authors', 'doi', 'label'], axis=1)
data.head()

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,"In December 2019, a new and highly contagious ...",0,0,0,1,0,0,1
1,"A novel coronavirus disease (COVID-19), transm...",0,0,0,0,1,0,1
2,"BACKGROUND: In December 2019, the novel corona...",1,0,0,0,0,0,0
3,During the coronavirus disease 2019 (COVID-19)...,0,0,0,0,1,0,0
4,OBJECTIVES: Sofosbuvir and daclatasvir are dir...,0,0,0,0,0,0,1


In [7]:
LABEL_COLUMNS = data.columns[1:]
Sample_row = data.iloc[1]
Sample_abstract = Sample_row.abstract
Sample_labels = Sample_row[LABEL_COLUMNS]

In [8]:
true_labels = [key for key, value in Sample_labels.to_dict().items() if value == 1]
true_labels

['Prevention', 'Treatment']

In [9]:
Sample_abstract

'A novel coronavirus disease (COVID-19), transmitted from humans to humans, has rapidly become the pandemic responsible for the current global health crisis. COVID-19 is caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which is said to be of zoonotic origin. This review describes the etiology and signs and symptoms as well as the current allopathic therapy for COVID-19. Additionally, findings of previous studies on the immunomodulatory effects and antiviral activities of particular foods and herbs on influenza virus and coronaviruses have been collated, with the aim of promoting the use of dietary therapy and herbal medicine as COVID-19 preventive therapies, while specific drugs and vaccines are yet to be discovered or are still under development. The volume of existing reports is irrefutable evidence that foods and herbs possess a potential antiviral ability against SARS-CoV-2 and can prevent COVID-19. Foods and herbs could be used as dietary or complementary th

In [11]:
encoding = tokenizer.batch_encode_plus(
    [Sample_abstract] + true_labels,
    return_tensors="pt",
    pad_to_max_length=True
) 

encoding.keys()

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
output = model(encoding["input_ids"], encoding["attention_mask"])[0]

In [15]:
sentence_rep = output[:1].mean(dim=1)
label_reps = output[1:].mean(dim=1)

In [17]:
similarities = F.cosine_similarity(sentence_rep, label_reps)
closest = similarities.argsort(descending=True)
for ind in closest:
    print(f'label: {true_labels[ind]} \t similarity: {similarities[ind]}')

label: Treatment 	 similarity: 0.24514493346214294
label: Prevention 	 similarity: 0.17372678220272064
